In [1]:
# code adapted from https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33#:~:text=The%20Keras%20ResNet%20got%20to,to%20do%20with%20weight%20initializations.
# import plaidml
# import plaidml.keras
# plaidml.install_backend()
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
NUM_SAMP = 1000

from PIL import Image
from glob import glob

# Clean Script
from glob import glob
import tensorflow as tf

from tensorflow.keras.utils import load_img

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,Model,load_model

# Collect paths to images based on label
nonCancerPaths = glob('imagedata/*/0/*.png')
cancerousPaths = glob('imagedata/*/1/*.png')

# Turn filepaths into image arrays to train a model
def paths_to_image(paths, label, num_samples):
    images = []
    labels = []
    for path in paths[0:num_samples]:
        image = load_img(path)
        image = image.resize([50, 50])
        imgArray = tf.keras.utils.img_to_array(image)
        images.append(imgArray)
        labels.append(label)
    return [images[0: num_samples], labels[0: num_samples]]
nonCancerImages = paths_to_image(nonCancerPaths, 0, NUM_SAMP)
cancerImages = paths_to_image(cancerousPaths, 1, NUM_SAMP)







2023-03-11 15:40:52.198586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.model_selection import train_test_split
allImages = nonCancerImages[0] + cancerImages[0]
allLabels = nonCancerImages[1] + cancerImages[1]

x_train, x_test, y_train, y_test = train_test_split(allImages, allLabels, test_size=.20, stratify= allLabels, random_state = 10)

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
# y_train = to_categorical(tf.keras.utils.to_categorical(y_train), num_classes = 2)
# y_test = to_categorical(tf.keras.utils.to_categorical(y_test), num_classes = 2)


print(np.array(x_train).shape)
print(np.array(y_train).shape)


model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')


(1600, 50, 50, 3)
(1600,)


2023-03-11 15:40:59.580108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(learning_rate=0.0001)
model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

print("FITTING")
history = model.fit(np.array(x_train), np.array(y_train), epochs=10, validation_data=(np.array(x_test), np.array(y_test)))







FITTING
Epoch 1/10
50/50 [==============================] - 48s 952ms/step - loss: 0.7110 - accuracy: 0.5219 - val_loss: 0.6688 - val_accuracy: 0.5975
Epoch 2/10
50/50 [==============================] - 48s 961ms/step - loss: 0.6621 - accuracy: 0.6194 - val_loss: 0.7795 - val_accuracy: 0.5300
Epoch 3/10
50/50 [==============================] - 48s 957ms/step - loss: 0.5638 - accuracy: 0.7450 - val_loss: 0.5436 - val_accuracy: 0.7950
Epoch 4/10
50/50 [==============================] - 49s 977ms/step - loss: 0.5373 - accuracy: 0.7675 - val_loss: 0.5437 - val_accuracy: 0.7925
Epoch 5/10
50/50 [==============================] - 51s 1s/step - loss: 0.4894 - accuracy: 0.8000 - val_loss: 0.4751 - val_accuracy: 0.8200
Epoch 6/10
50/50 [==============================] - 50s 1s/step - loss: 0.4304 - accuracy: 0.8163 - val_loss: 0.4272 - val_accuracy: 0.8275
Epoch 7/10
13/50 [======>.......................] - ETA: 34s - loss: 0.3847 - accuracy: 0.8317

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


In [ ]:
model.summary()